In [ ]:
import tensorflow as tf
import numpy as np               # 陣列運算的數學函數函式庫
import matplotlib.pyplot as plt  # 繪圖庫
import os                        # 處理文件和目錄
import gzip

In [ ]:
# 建立 10 種類別
class_names = ['T-shirt/top',  # 0: index
               'Trouser',      # 1
               'Pullover',     # 2
               'Dress',        # 3
               'Coat',         # 4
               'Sandal',       # 5
               'Shirt',        # 6
               'Sneaker',      # 7
               'Bag',          # 8
               'Ankle boot']   # 9

In [ ]:
# 建立 Function, using the def keyword:
# 秀單張圖
def plot_image(img):  # 參攷[1]
    plt.figure()      # 建立一個Figure顯示區, 參攷[2]
    plt.imshow(img)   # 顯示圖片, 參攷[3]
    # plt.colorbar()
    plt.grid(True)    # 顯示網格, 參攷[4]
    plt.show()        # 秀圖, 參攷[5]

#### 參攷:
1. https://docs.python.org/3/tutorial/controlflow.html#defining-functions
2. https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.figure.html
3. https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.imshow.html?highlight=imshow#matplotlib.pyplot.imshow
4. https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.grid.html?highlight=grid#matplotlib.pyplot.grid
5. https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.show.html?highlight=show#matplotlib.pyplot.show

In [ ]:
# 秀出多張訓練集圖片(含label)
def plot_images(imgs, lbls): 
    plt.figure(figsize=(15, 15))  # 定義一個圖像視窗, 每個小視窗大小(寬,高), 單位為英吋
    for i in range(100):
        plt.subplot(10, 10, i + 1)  # 分格顯示, subplot(numRows=行, numCols=列, plotNum=編號) 
        plt.xticks([])              # 無 x軸刻度
        plt.yticks([])              # 無 y軸刻度
        plt.grid(False)             # 無 格線
        plt.imshow(imgs[i], cmap=plt.cm.binary)  # colormaps=gray將影像顯示為黑底白字、gray_r將其顯示為白底黑字
        plt.xlabel(class_names[lbls[i]], fontsize=10)         # 設定 x-axis 標籤 秀 class name, 參攷[1]

    plt.tight_layout()  # 參攷[2]
    plt.show()

#### 參攷:
1. https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.xlabel.html?highlight=xlabel#matplotlib.pyplot.xlabel
2. https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.tight_layout.html

In [ ]:
# 定義繪圖函式
def plot_Images(imgs, labels=None, rows=1, figsize=(20, 8), fontsize=10):
    figure = plt.figure(figsize=figsize)
    cols = max(1, len(imgs) // rows - 1)
    labels_present = False

    # checking if labels is a numpy array
    if type(labels).__module__ == np.__name__:
        labels_present = labels.any()
    elif labels:
        labels_present = True

    for i in range(len(imgs)):
        subplot = figure.add_subplot(rows, cols + 1, i + 1)

        # axis off, but leave a bounding box
        plt.tick_params(   # 設置參數, 參攷
            axis='both',   # 設置X軸、Y軸
            which='both',  # 設置主刻度線、副刻度線
            bottom=False,  # 設置繪圖區四個邊框線上的的刻度線是否顯示
            top=False,
            left=False,
            right=False,
            labelbottom=False,  # 設置繪圖區四個邊框線上的刻度線標籤是否顯示
            labelleft=False)

        # plot labels if present
        if labels_present:
            subplot.set_title(class_names[labels[i]], fontsize=fontsize)
        plt.imshow(imgs[i][:,:,0], cmap='binary')

    plt.tight_layout()
    plt.show()

#### 參攷: https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.tick_params.html

In [ ]:
# 繪製預測(含label)的圖片
def plot_pred_image(i, predictions_array, true_label, img):
    predictions_array, true_label, img= predictions_array, true_label[i], img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img, cmap=plt.cm.binary)

    predicted_label = np.argmax(predictions_array)
    if predicted_label == true_label:
        color = 'blue'  # 相等
    else:
        color = 'red'   # 不相等

    plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100 * np.max(predictions_array),  # 參攷
                                class_names[true_label]),
                                color=color)

#### 參攷: https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.max.html

In [ ]:
# 繪製測試label vs. 真實label的bar圖
def plot_value_array(i, predictions_array, true_label):
    predictions_array, true_label = predictions_array, true_label[i]
    plt.grid(False)
    plt.xticks(range(10))
    plt.yticks([])
    thisplot = plt.bar(range(10), predictions_array, color="#777777")  # 參攷
    plt.ylim([0, 1])
    predicted_label = np.argmax(predictions_array)

    thisplot[predicted_label].set_color('red')
    thisplot[true_label].set_color('blue')

#### 參攷: https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.bar.html

In [ ]:
# 計算saved_model目錄中variables的size
def get_size(model_dir):
    print(model_dir)
    print("")

    pb_size = os.path.getsize(os.path.join(model_dir, 'saved_model.pb'))  # 參攷

    variables_size = 0
    if os.path.exists(os.path.join(model_dir, 'variables/variables.data-00000-of-00002')):
        variables_size = os.path.getsize(os.path.join(model_dir, 'variables/variables.data-00001-of-00002'))
        variables_size += os.path.getsize(os.path.join(model_dir, 'variables/variables.index'))

    print("Model size: {} KB".format(round(pb_size / (1024.0), 3)))
    print("Variables size: {} KB".format(round(variables_size / (1024.0), 3)))
    print("Total Size: {} KB".format(round((pb_size + variables_size) / (1024.0), 3)))

#### 參攷: https://docs.python.org/3/library/os.path.html#os.path.getsize

In [ ]:
# 繪製預測圖片(含label)
def plot_predict(predict, labels, images):
    num_rows = 5
    num_cols = 3
    num_images = num_rows * num_cols
    plt.figure(figsize=(2 * 2 * num_cols, 2 * num_rows))

    for i in range(num_images):
        plt.subplot(num_rows, 2 * num_cols, 2 * i + 1)  # 參攷
        plot_pred_image(i, predict[i], labels, images)
        plt.subplot(num_rows, 2 * num_cols, 2 * i + 2)
        plot_value_array(i, predict[i], labels)

    plt.tight_layout()
    plt.show()

#### 參攷: https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.subplot.html

In [ ]:
# 讀取路徑圖片並正規化
def load_and_preprocess_image_rgb(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(contents=image, channels=3)  # RGB, 參攷
    image = tf.image.resize(image, size=[256, 256])
    # Normalize the pixel values
    image /= 255.0  # normalize to [0,1] range
    return image

#### 參攷: https://www.tensorflow.org/api_docs/python/tf/io/decode_jpeg

In [ ]:
# 讀取路徑圖片並預處理
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)  # 參攷
    return preprocess_image(image)

#### 參攷: https://www.tensorflow.org/api_docs/python/tf/io/read_file

In [ ]:
# 圖片預處理, size=1,000x1,000=>256x256, channels=3=>1
def preprocess_image(image):
    # image = tf.image.decode_jpeg(contents=image, channels=3)  # RGB
    image = tf.image.decode_jpeg(contents=image, channels=1)    # grayscale, only .jpg
    image = tf.image.resize(image, size=[256, 256])  # 參攷
    # Normalize the pixel values
    image /= 255.0  # normalize to [0,1] range
    return image

In [ ]:
# 讀取路徑圖片並預處理, 28x28
def load_and_preprocess_image_28(path):
    image = tf.io.read_file(path)  # 參攷
    return preprocess_image_28(image)

In [ ]:
# 圖片預處理, size=1,000x1,000=>256x256, channels=3=>1
def preprocess_image_28(image):
    # image = tf.image.decode_jpeg(contents=image, channels=3)  # RGB
    image = tf.image.decode_jpeg(contents=image, channels=1)    # grayscale, only .jpg
    image = tf.image.resize(image, size=[28, 28])  # 參攷
    # Normalize the pixel values
    image /= 255.0  # normalize to [0,1] range
    return image

#### 參攷: https://www.tensorflow.org/api_docs/python/tf/image/resize

In [ ]:
# 載入及預處理 label
def load_and_preprocess_from_path_label(path, label):
    return load_and_preprocess_image(path), label

In [ ]:
# 載入及預處理 label
def load_and_preprocess_from_path_label_28(path, label):
    return load_and_preprocess_image_28(path), label

In [ ]:
# 秀單張圖
def show_img(img_path, label, label_names):
    plt.imshow(load_and_preprocess_image_rgb(img_path))
    plt.grid(False)
    plt.xlabel(label_names[label] + '=' + str(label))  # 參攷
    # plt.title(label_names[label].title())
    plt.show()

#### 參攷: https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.xlabel.html

In [ ]:
# 繪製模型訓練history的trend圖
def history_plot(history, key):
    plt.plot(history.history[key])
    plt.plot(history.history['val_' + key])
    plt.title('Model ' + key)
    plt.ylabel(key)
    plt.xlabel('epoch')
    if key == 'loss':
        locs='upper right'
    else:
        locs='upper left'
    
    plt.legend(['train', 'test'], loc=locs)  # 參攷
    
    plt.show()

#### 參攷: https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.legend.html

In [ ]:
# 解壓.gz
def un_gz(file_name):
    """ ungz zip file """
    f_name = file_name.replace(".gz", "")
    # 去掉文件的名稱
    g_file = gzip.GzipFile(file_name)
    # 創建gzip對象
    open(f_name, "wb+").write(g_file.read())
    # gzip對象用read()打開後，寫入open()建立的文件裡
    g_file.close()
    # 關閉gzip對象